<a href="https://colab.research.google.com/github/tmdang1101/amazon_product_recommender_system/blob/main/Model_1_Collaborative_Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Import libraries
import numpy as np
import pandas as pd
import scipy.stats as ss
import seaborn as sns
import matplotlib.pyplot as plt

import gzip
import json

In [6]:
# Connects this notebook to Google Drive
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

pathname = "/content/drive/My Drive/Recommender System/"
os.chdir(pathname)

Mounted at /content/drive


In [7]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF('/content/drive/My Drive/Recommender System/Data/Movies_and_TV_5.json.gz')

In [8]:
df.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5.0,True,"11 9, 2012",A2M1CU2IRZG0K9,0005089549,{'Format:': ' VHS Tape'},Terri,So sorry I didn't purchase this years ago when...,Amazing!,1352419200,NaN,NaN
1,5.0,True,"12 30, 2011",AFTUJYISOFHY6,0005089549,{'Format:': ' VHS Tape'},Melissa D. Abercrombie,Believe me when I tell you that you will recei...,Great Gospel VHS of the Cathedrals!,1325203200,NaN,NaN
2,5.0,True,"04 21, 2005",A3JVF9Y53BEOGC,000503860X,{'Format:': ' DVD'},Anthony Thompson,"I have seen X live many times, both in the ear...",A great document of a great band,1114041600,11,NaN
3,5.0,True,"04 6, 2005",A12VPEOEZS1KTC,000503860X,{'Format:': ' DVD'},JadeRain,"I was so excited for this! Finally, a live co...",YES!! X LIVE!!,1112745600,5,NaN
4,5.0,True,"12 3, 2010",ATLZNVLYKP9AZ,000503860X,{'Format:': ' DVD'},T. Fisher,X is one of the best punk bands ever. I don't ...,X have still got it,1291334400,5,NaN


In [9]:
df.shape

(3410019, 12)

In [10]:
ss.describe(df['overall'])

DescribeResult(nobs=3410019, minmax=(1.0, 5.0), mean=4.221320174462371, variance=1.360619309461769, skewness=-1.4843835182448823, kurtosis=1.2016060314839265)

In [11]:
metadata = getDF('/content/drive/My Drive/Recommender System/Data/meta_Movies_and_TV.json.gz')

In [12]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,"[Movies & TV, Movies]",,[],,Understanding Seizures and Epilepsy,[],,,[],"886,503 in Movies & TV (",[],Movies & TV,,,,0000695009,[],[],NaN
1,"[Movies & TV, Movies]",,[],,Spirit Led&mdash;Moving By Grace In The Holy S...,[],,,[],"342,688 in Movies & TV (",[],Movies & TV,,,,0000791156,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
2,"[Movies & TV, Movies]",,[Disc 1: Flour Power (Scones; Shortcakes; Sout...,,My Fair Pastry (Good Eats Vol. 9),[],,Alton Brown,[],"370,026 in Movies & TV (",[],Movies & TV,,,,0000143529,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
3,"[Movies & TV, Movies]",,[Barefoot Contessa Volume 2: On these three di...,,"Barefoot Contessa (with Ina Garten), Entertain...","[B002I5GNW4, B005WXPVMM, B009UY3W8O, B00N27ID1...",,Ina Garten,[],"342,914 in Movies & TV (","[B002I5GNW4, 0804187045, B009UY3W8O, 060960219...",Movies & TV,,,$74.95,0000143588,[],[],NaN
4,"[Movies & TV, Movies]",,[Rise and Swine (Good Eats Vol. 7) includes bo...,,Rise and Swine (Good Eats Vol. 7),"[B000P1CKES, B000NR4CRM]",,Alton Brown,[],"351,684 in Movies & TV (",[B0015SVNXY],Movies & TV,,,,0000143502,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN


In [13]:
metadata.shape

(203766, 19)

In [15]:
list(metadata['title']).index('Harry Potter and the Prisoner of Azkaban')

26876

In [16]:
metadata.loc[26876]

category           [Movies & TV, Characters & Series, Harry Potte...
tech1                                                               
description        [As Harry Potter readies himself for his third...
fit                                                                 
title                       Harry Potter and the Prisoner of Azkaban
also_buy           [B000E6EK2Y, B01KKN0HYU, B002PMV9FG, B000W7F5S...
tech2                                                               
brand                                               Daniel Radcliffe
feature                                                           []
rank                                         32,077 in Movies & TV (
also_view          [B01KKN0HZ4, B01KKN0HYU, B000E6EK2Y, B01KW24RN...
main_cat                                                 Movies & TV
similar_item                                                        
date                                                                
price                             

In [17]:
metadata.loc[26876]['also_buy']

['B000E6EK2Y',
 'B01KKN0HYU',
 'B002PMV9FG',
 'B000W7F5SS',
 'B001UV4XHY',
 'B001UV4XIS',
 'B01KKN0I1C',
 'B07CQP6R9Y',
 'B008UZA8FY',
 'B01GWBPWTO',
 'B07CPC394C',
 'B004WO6WAM',
 'B01GWC5TFU',
 'B00UGQI3GG',
 'B01KKN0I9E',
 'B01KW24REK',
 'B073LC2F8S',
 'B07CPF532R',
 'B01GWC6RGU',
 '0439139600',
 'B01KW24RNQ',
 '0439139597',
 'B01KKN0HZ4',
 '0439136369',
 'B01KKN0I6W',
 'B01KKN0I4Y',
 'B07CXZ9C5B',
 'B004WO6HT8',
 '6304698623',
 'B000NVT0TI',
 'B01KW24RC2',
 'B01B80CM4W',
 'B00006HBUJ',
 '0545791340',
 'B00HE7YSKA',
 'B00009TB5G',
 '0439358078',
 'B00003CX5P',
 '0783225547',
 '0439136350',
 'B00HWWUQWQ',
 'B000VBJEEG',
 'B075FSRZK8',
 'B008UZDXEW',
 'B07CQKS23B',
 'B01I5IPP4E',
 'B07BKM4N9B',
 '1549697137',
 'B00007JMD8',
 'B07G1THXKY',
 'B00HFFE8HE',
 'B01MTF4482',
 'B000Y11B7G',
 'B00005JN4W',
 '1510729313',
 'B004A8ZWW4',
 'B079KJLDF2',
 'B07DQMJBHS',
 'B07DL6X6ZJ',
 'B001QOGYBI',
 '0545139708',
 '1338132326',
 'B06XJKXY4R',
 'B06ZYGQBQV',
 '133821666X',
 'B07G2MYV7Z',
 'B01MQMCI

In [18]:
metadata.loc[26876]['also_view']

['B01KKN0HZ4',
 'B01KKN0HYU',
 'B000E6EK2Y',
 'B01KW24RNQ',
 'B000W7F5SS',
 'B002PMV9FG',
 'B01KMWAV3Q',
 'B01KMWAVM2',
 'B008UZ949K',
 'B01KKN0I1C',
 'B07CPMYR82',
 'B07CQP6R9Y',
 'B008UZA8FY',
 'B01GWBPWTO',
 'B00H6T0NZA',
 'B00UGQI3GG',
 'B005OCFGTO',
 'B07CPC394C',
 'B00UB1MP92',
 'B01C45OBKI',
 'B001DPHDJ2',
 'B004WO6WAM',
 'B0002TV2WY',
 'B01GWC5TFU',
 'B008UZDXEW',
 'B008UZ7N3E',
 'B01C45O9YG',
 'B007FSEAHY',
 'B00H6TARD8',
 'B07FSG5XT7',
 'B00005JNJV',
 'B0771HW47S',
 'B002ZG980U',
 'B004LWZWGA',
 'B002Q4VPM0',
 'B01GWC6RGU',
 'B01LTHOAGC',
 'B004LRPCSS',
 'B076X2YHRF',
 'B01C45O8S8',
 'B073LWR8XD',
 'B00YZH1PEY',
 'B01N7BLIK1',
 'B000VWYJ86',
 'B01C45O9F0',
 'B01JPYUZFQ',
 'B01HOIXOHQ',
 'B073WYD3CH',
 'B000E33W26',
 'B00UGPH2PA',
 'B01N47BB16',
 'B0013FDM7E',
 '0800195175',
 'B000E33W1W',
 'B00L04WB20',
 'B07223GML6']

In [19]:
metadata.loc[26876]['similar_item']

''

In [20]:
relevant_items = metadata.loc[26876]['also_buy'] + metadata.loc[26876]['also_view']

In [21]:
# Main Training Set
ratings = df[['reviewerID','asin','overall']]

In [22]:
ratings.head()

,reviewerID,asin,overall
0,A2M1CU2IRZG0K9,0005089549,5.0
1,AFTUJYISOFHY6,0005089549,5.0
2,A3JVF9Y53BEOGC,000503860X,5.0
3,A12VPEOEZS1KTC,000503860X,5.0
4,ATLZNVLYKP9AZ,000503860X,5.0


In [23]:
ratings.shape

(3410019, 3)

In [24]:
#ratings_with_name = ratings.merge(metadata, on='asin')[['reviewerID','asin','title','overall']]

In [25]:
#ratings_with_name.head()

In [26]:
#ratings_with_name.shape

In [27]:
#ratings_with_name.isnull().sum()

In [28]:
#ratings_with_name.duplicated().sum()

In [29]:
#ratings_with_name = ratings_with_name.drop_duplicates()

In [30]:
#ratings_with_name.duplicated().sum()

In [31]:
ratings.rename(columns={'reviewerID':'user_id'}, inplace=True)
ratings.rename(columns={'asin':'product_id'}, inplace=True)
ratings.rename(columns={'overall':'rating'}, inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [32]:
ratings.head()

,user_id,product_id,rating
0,A2M1CU2IRZG0K9,0005089549,5.0
1,AFTUJYISOFHY6,0005089549,5.0
2,A3JVF9Y53BEOGC,000503860X,5.0
3,A12VPEOEZS1KTC,000503860X,5.0
4,ATLZNVLYKP9AZ,000503860X,5.0


In [33]:
# Creates Sparse Matrix as the Input
from scipy.sparse import csr_matrix

def create_X(df):
    """
    Generates a sparse matrix from ratings dataframe.
    
    Args:
        df: pandas dataframe
    
    Returns:
        X: sparse matrix
        user_mapper: dict that maps user id's to user indices
        user_inv_mapper: dict that maps user indices to user id's
        movie_mapper: dict that maps movie id's to movie indices
        movie_inv_mapper: dict that maps movie indices to movie id's
    """
    N = df['user_id'].nunique()
    M = df['product_id'].nunique()

    user_mapper = dict(zip(np.unique(df["user_id"]), list(range(N))))
    movie_mapper = dict(zip(np.unique(df["product_id"]), list(range(M))))
    
    user_inv_mapper = dict(zip(list(range(N)), np.unique(df["user_id"])))
    movie_inv_mapper = dict(zip(list(range(M)), np.unique(df["product_id"])))
    
    user_index = [user_mapper[i] for i in df['user_id']]
    movie_index = [movie_mapper[i] for i in df['product_id']]

    X = csr_matrix((df["rating"], (movie_index, user_index)), shape=(M, N))
    
    return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper

In [34]:
X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_X(ratings)

In [35]:
X

<60175x297529 sparse matrix of type '<class 'numpy.float64'>'
	with 3282379 stored elements in Compressed Sparse Row format>

In [36]:
sparsity = X.count_nonzero()/(X.shape[0]*X.shape[1])

print(f"Matrix sparsity: {round(sparsity*100,2)}%")

Matrix sparsity: 0.02%


In [37]:
from sklearn.neighbors import NearestNeighbors

def find_similar_movies(movie_id, X, k, metric='cosine', show_distance=False):
    """
    Finds k-nearest neighbours for a given movie id.
    
    Args:
        movie_id: id of the movie of interest
        X: user-item utility matrix
        k: number of similar movies to retrieve
        metric: distance metric for kNN calculations
    
    Returns:
        list of k similar movie ID's
    """
    neighbour_ids = []
    
    movie_ind = movie_mapper[movie_id]
    movie_vec = X[movie_ind]
    k+=1
    kNN = NearestNeighbors(n_neighbors=k, algorithm="brute", metric=metric)
    kNN.fit(X)
    if isinstance(movie_vec, (np.ndarray)):
        movie_vec = movie_vec.reshape(1,-1)
    neighbour = kNN.kneighbors(movie_vec, return_distance=show_distance)
    for i in range(0,k):
        n = neighbour.item(i)
        neighbour_ids.append(movie_inv_mapper[n])
    neighbour_ids.pop(0)
    return neighbour_ids

In [52]:
#movie_titles = dict(zip(ratings['movieId'], ratings['title']))

#movie_id = 'B00IT70F3S'     # Avengers: Age of Ultron
#movie_id = 'B00LYYUTEI'    # The Dark Knight
movie_id = 'B00005JMAH'     # Harry Potter and the Prisoner of Azkaban

similar_ids = find_similar_movies(movie_id, X, k=20)
#movie_title = movie_titles[movie_id]

print(f"Because you watched {movie_id}")
for i in similar_ids:
    print(i)

Because you watched B00005JMAH
B000E6EK3S
B00005JPI2
B000ZECQ08
B001UV4XI8
B001UV4XIS
B00005JMQW
B00005JMQZ
B00005JKZY
B00005JNJV
7799146915
B00064LJVO
B00Z9YZRVE
B00003CWT6
B00005JKZU
7883704591
B01AUH49W2
B00005JM5E
B00009W2GQ
B00005JNTU
B0002KVUKW


In [47]:
def similar_movies(ids):
  for id in ids:
    print(metadata.loc[metadata['asin'] == id]['title'])

In [53]:
similar_movies(similar_ids)

79415    Harry Potter and the Goblet of Fire
Name: title, dtype: object
27164    Harry Potter and the Order of the Phoenix
49091    Harry Potter and the Order of the Phoenix
Name: title, dtype: object
105857    Harry Potter and the Half-Blood Prince
Name: title, dtype: object
127394     Harry Potter and the Deathly Hallows, Part 1 ...
Name: title, dtype: object
127372    HP7: Deathly Hallows, P2 (DVD)
Name: title, dtype: object
26936    Spider-Man 2
48863    Spider-Man 2
Name: title, dtype: object
26926    Shrek 2
48853    Shrek 2
Name: title, dtype: object
26792    The Lord of the Rings: The Return of the King
48719    The Lord of the Rings: The Return of the King
Name: title, dtype: object
26992    Batman Begins
48919    Batman Begins
Name: title, dtype: object
19337    Star Wars: Episode III - Revenge of the Sith (...
Name: title, dtype: object
65902    The Village: Vista Series
Name: title, dtype: object
195787    The Lord Of The Rings: The Return Of The King
Name: title, dtype: ob

In [54]:
similar_movies(relevant_items)

Series([], Name: title, dtype: object)
Series([], Name: title, dtype: object)
Series([], Name: title, dtype: object)
Series([], Name: title, dtype: object)
Series([], Name: title, dtype: object)
127372    HP7: Deathly Hallows, P2 (DVD)
Name: title, dtype: object
Series([], Name: title, dtype: object)
Series([], Name: title, dtype: object)
163017    HP Double Feature: Year 7 (DBFE) (DVD)
Name: title, dtype: object
Series([], Name: title, dtype: object)
Series([], Name: title, dtype: object)
Series([], Name: title, dtype: object)
Series([], Name: title, dtype: object)
193564    Harry Potter and the Order of the Phoenix (Two...
Name: title, dtype: object
Series([], Name: title, dtype: object)
Series([], Name: title, dtype: object)
Series([], Name: title, dtype: object)
Series([], Name: title, dtype: object)
Series([], Name: title, dtype: object)
Series([], Name: title, dtype: object)
Series([], Name: title, dtype: object)
Series([], Name: title, dtype: object)
Series([], Name: title, dtyp

In [50]:
def hits(ids):
  hit = 0
  for id in ids:
    if id in relevant_items:
      hit += 1
  print(hit)

In [55]:
hits(similar_ids)

2


In [44]:
relevant_items

['B000E6EK2Y',
 'B01KKN0HYU',
 'B002PMV9FG',
 'B000W7F5SS',
 'B001UV4XHY',
 'B001UV4XIS',
 'B01KKN0I1C',
 'B07CQP6R9Y',
 'B008UZA8FY',
 'B01GWBPWTO',
 'B07CPC394C',
 'B004WO6WAM',
 'B01GWC5TFU',
 'B00UGQI3GG',
 'B01KKN0I9E',
 'B01KW24REK',
 'B073LC2F8S',
 'B07CPF532R',
 'B01GWC6RGU',
 '0439139600',
 'B01KW24RNQ',
 '0439139597',
 'B01KKN0HZ4',
 '0439136369',
 'B01KKN0I6W',
 'B01KKN0I4Y',
 'B07CXZ9C5B',
 'B004WO6HT8',
 '6304698623',
 'B000NVT0TI',
 'B01KW24RC2',
 'B01B80CM4W',
 'B00006HBUJ',
 '0545791340',
 'B00HE7YSKA',
 'B00009TB5G',
 '0439358078',
 'B00003CX5P',
 '0783225547',
 '0439136350',
 'B00HWWUQWQ',
 'B000VBJEEG',
 'B075FSRZK8',
 'B008UZDXEW',
 'B07CQKS23B',
 'B01I5IPP4E',
 'B07BKM4N9B',
 '1549697137',
 'B00007JMD8',
 'B07G1THXKY',
 'B00HFFE8HE',
 'B01MTF4482',
 'B000Y11B7G',
 'B00005JN4W',
 '1510729313',
 'B004A8ZWW4',
 'B079KJLDF2',
 'B07DQMJBHS',
 'B07DL6X6ZJ',
 'B001QOGYBI',
 '0545139708',
 '1338132326',
 'B06XJKXY4R',
 'B06ZYGQBQV',
 '133821666X',
 'B07G2MYV7Z',
 'B01MQMCI

In [45]:
metadata.loc[metadata['asin'] == 'B07448VCM1']['title']

Series([], Name: title, dtype: object)